# Generate sequences for the goal selection experiment

## Import statements

In [1]:
import numpy as np
import json
import os
from itertools import combinations


## Utility functions

In [2]:
def combs_to_array(iterable, r, fun=combinations):
    ''' 
    Return itertools combinations (or permutations) as an array of arrays
    '''
    return(np.array([np.array(comb) for comb in fun(iterable, r)]))


def shuffled(arr):
    ''' 
    Return a shuffled array
    '''
    arr_shuffled = arr.copy()
    np.random.shuffle(arr_shuffled)
    return(arr_shuffled)


def is_arr_in_list(arr, ls):
    ''' 
    Check if an array is in a list/array of arrays
    '''
    return np.any(np.all(arr == ls, axis=1))


def get_keys_by_value(dictionary, target_value):
    ''' 
    Return the keys in a dictionary that corresponds to a target value
    '''
    keys = []
    for key, value in dictionary.items():
        if np.all(value == target_value):
            keys.append(key)
    if keys:
        return keys if len(keys)>1 else keys[0]
    else:
        return None  # Return None if the target value is not found in any list
    

def reordered_dict(item):
    ''' 
    Return a dictionary sorted by keys
    '''
    return {k: reordered_dict(v) if isinstance(v, dict) else v for k, v in sorted(item.items())}
 

def make_json_compatible(new_dict):
    for key_i in new_dict.keys():
        if type(new_dict[key_i]) == dict:
            if type(list(new_dict[key_i].values())[0]) == list:
                new_dict[key_i] = {str(key_j): [int(k) for k in val_j] for key_j, val_j in new_dict[key_i].items()} 
            else:
                new_dict[key_i] = {str(key_j): int(val_j) for key_j, val_j in new_dict[key_i].items()} 
        elif type(new_dict[key_i]) == list:
            new_dict[key_i] = [int(val_i) for val_i in new_dict[key_i]]
    return new_dict

## Main functions

In [3]:
def get_training_seq(n_train, n_goals=6):
    assert n_train % 6 == 0, f"n_train ({n_train}) cannot evenly divided by n_goals ({n_goals})"
    training_seq = np.zeros(n_train)
    for i in range(n_train//n_goals):
        training_seq[i*n_goals:(i+1)*n_goals] = np.random.permutation(n_goals)
    return [int(i) for i in training_seq]


def add_goal_info(base_dict, jsonify=True):
    n_goals, n_potions, n_ingredients = 6, 4, 4
    new_dict = base_dict.copy()

    # goal_sprite_order will determine where each goal goes
    goal_sprite_order = np.random.permutation(np.arange(6))
    # potion0 is the empty potion, so sprite indices start at 1
    # there are 9 different potions
    all_goal_sprites = shuffled(np.arange(1, 10))
    goal_sprite_indices = all_goal_sprites[[0, 1, 2, 3, 2, 3]]

    # goalIdToPos = for each goal id (0 is basic1, 1 is basic2, 3 is composite1, etc.), what should its position on the board be?
    new_dict["goalIdToPos"] = {i: goal_sprite_order[i] for i in range(n_goals)}

    # goalIdToSprite = for each goal id, what should its sprite be?
    new_dict["goalIdToSprite"] = {i: goal_sprite_indices[i] for i in range(n_goals)}

    # goalSpriteIdx = for each goal position on the board, what should its sprite be?
    new_dict["goalSpriteIdx"] = reordered_dict({val: new_dict["goalIdToSprite"][key] for key, val in new_dict["goalIdToPos"].items()})
    # for key, val in new_dict["goalIdToPos"].items():
    #     print(key, val)
    #     new_dict["goalSpriteIdx"][val] = new_dict["goalIdToSprite"][key]
    # new_dict["goalSpriteIdx"] = reordered_dict(new_dict["goalSpriteIdx"])                    

    # ingredients 0-3 are the basic ingredients, after that it's potions
    basic_ingr_sprite_order = list(shuffled(np.arange(n_ingredients)))
    potion_ingr_sprite_order = list(shuffled(np.arange(n_ingredients)))
    # potions that don't correspond to goals
    additional_potions = np.random.choice(all_goal_sprites[4:], n_ingredients//2, replace=False) + (n_ingredients-1)
    # potions that also correspond to goals
    reused_potions = [new_dict["goalIdToSprite"][0]+n_ingredients-1, new_dict["goalIdToSprite"][1]+n_ingredients-1]
    
    # all ingredient potions
    ingr_potions = np.hstack((reused_potions, additional_potions))
    ingr_potions = ingr_potions[potion_ingr_sprite_order]
    # ingredientSpritesIdx = for each goal, what ingredients shoudld be available?
    for goal_id, goal_pos in new_dict["goalIdToPos"].items():
        new_dict["ingredientSpritesIdx"][goal_pos] = basic_ingr_sprite_order if goal_id < 4 else list(ingr_potions)
    new_dict["ingredientSpritesIdx"] = reordered_dict(new_dict["ingredientSpritesIdx"])

    # solutions
    sols = [[] for i in range(n_goals)]
    # first two potions: take numbers between 0 and n_ingredients and distribute them across two goals
    sols[0:n_potions//2] = np.split(np.random.permutation(n_ingredients), n_potions//2)[0:n_potions//2]
    # third potion: (with shuffling) is first + second or second + first
    sols[2] = np.hstack(shuffled([sols[0], sols[1]]))
    # fourth potion: a random combination that doesn't contain the solution for any of the first two potions 
    # (even in a different order)
    sorted_first_two = np.sort(sols[0:n_potions//2], axis=1)
    valid_comb = False
    while not valid_comb:
        new_comb = np.random.permutation(n_ingredients)
        if not (is_arr_in_list(np.sort(new_comb[0:2]), sorted_first_two) or \
                is_arr_in_list(np.sort(new_comb[2:]), sorted_first_two)):
            valid_comb = True
            sols[3] = new_comb
    new_dict["goalSolutions"] = reordered_dict({goal_sprite_order[i]:list(sol) for i, sol in enumerate(sols)})
    # fifth and sixth potions are made from other potions
    if n_goals > 4:
        # fifth potion based on whether the third is frist + second or second + first
        # which goes first in sols[2]? the first potion's sequence (0) or the second (1)?
        which_first = 0 if np.all(sols[2][0:n_ingredients//2] == sols[0]) else 1
        sols[4] = [which_first, 1-which_first]
        # sixth potion is the remaining two potions
        sols[5] = shuffled([2, 3])
    # add to the solutions dictionary
    new_dict["goalSolutions"][new_dict["goalIdToPos"][4]] = [potion_ingr_sprite_order.index(i) for i in sols[4]]
    new_dict["goalSolutions"][new_dict["goalIdToPos"][5]] = [potion_ingr_sprite_order.index(i) for i in sols[5]]

    if jsonify:
        new_dict = make_json_compatible(new_dict)

    return new_dict


## Initialize variables

In [4]:
n_seqs = 10

store_data = True

n_blocks = 3
n_training_blocks = 1  # how many of the blocks are training blocks?
n_trials = 6*n_blocks           # total, including training
n_trials_per_block = n_trials//n_blocks

iti_dur_pre = 0.5
iti_dur_post = 1.8
fb_dur = 2

# Initialize a basic dictionary with common information across sequences
base_dict = {
    "numBlocks": n_blocks,
    "numTrainingBlocks": n_training_blocks,
    "numTrialsInBlock": n_trials_per_block,
    "itiDurPre": iti_dur_pre,
    "itiDurPost": iti_dur_post,
    "feedbackDur": fb_dur,
    "storeData": store_data,

    "trainingSeq": [],

    "goalSpriteIdx": {},
    "ingredientSpritesIdx": {},
    "goalSolutions": {},
    "goalIdToPos": {},
    "goalIdToSprite": {}
 }


## Get sequences

In [6]:
for seq_idx in range(n_seqs):
    new_dict = add_goal_info(base_dict)
    new_dict["trainingSeq"] = get_training_seq(n_trials_per_block*n_training_blocks)
    json_object = json.dumps(new_dict, indent=2) 
    with open(f"{os.path.join(os.getcwd(), '../')}config{seq_idx}.json", "w") as outfile:
        outfile.write(json_object)